In [41]:
import pandas as pd
import numpy as np
from obspy.clients.fdsn import Client
from pnwstore.mseed import WaveformClient
from datetime import datetime
import obspy
from matplotlib import pyplot as plt
from obspy import UTCDateTime

from obspy.geodetics import locations2degrees, degrees2kilometers
from tqdm import tqdm

In [42]:
# Read Morton's catalog
events_morton = pd.read_csv('../data/ds01.csv')

In [43]:
# Convert the TSTRING to datetime
events_morton['datetime'] = pd.to_datetime(events_morton['TSTRING'], format='%Y%m%d%H%M%S', utc=True)

In [44]:
# Get the events in the Morton catalog that occurred on 10/14/2012
t1 = pd.Timestamp('2012-10-14 00:00:00.000000+0000', tz='UTC')
t2 = pd.Timestamp('2012-10-14 23:59:59.999999+0000', tz='UTC')

events_morton_2021014= events_morton.loc[(events_morton['datetime'] > t1) & (events_morton['datetime'] < t2) ]

In [45]:
events_morton_2021014

,CI YEAR,TSTRING,YEAR,MONTH,DAY,HOUR,MINUTE,SECOND,LAT,LON,...,dist to nearest stn,tt RMS,ERH,ERZ,STRIKE,DIP,RAKE,PLATE DESIGNATION,TEMPLATE EVENT?,datetime
671,2.0,2.012101e+13,2012.0,10.0,14.0,2.0,39.0,44.31,40.2517,-124.4555,...,4.7,0.25,0.9,1.6,NaN,NaN,NaN,Seaward of Trench,NaN,2012-10-14 02:39:44+00:00
672,2.0,2.012101e+13,2012.0,10.0,14.0,3.0,8.0,1.10,40.7087,-124.1907,...,18.2,0.24,1.0,1.6,NaN,NaN,NaN,Slab,NaN,2012-10-14 03:08:01+00:00
673,2.0,2.012101e+13,2012.0,10.0,14.0,16.0,48.0,33.63,40.3390,-124.6048,...,13.2,0.07,5.4,12.3,NaN,NaN,NaN,Undef,NaN,2012-10-14 16:48:33+00:00


In [46]:
# Load all the pick assignments 
mycatalog = pd.read_csv('../data/all_pick_assignments_october.csv')
# Convert the time series in all_pick_assignments to datetime
mycatalog['datetime'] = pd.to_datetime(mycatalog['time'], utc = True)
# Print the first five rows of the catalog
mycatalog.head()

,Unnamed: 0,idx,time,x,y,z,picks,latitude,longitude,depth,event_idx,pick_idx,residual,station,phase,time_pick,datetime
0,0,4,2012-10-01 00:57:24.351689+00:00,7.588919,82.706331,0.390625,6,42.494557,-124.657684,0.390625,4,67646,0.684967,FS13B,P,1.349053e+09,2012-10-01 00:57:24.351689+00:00
1,1,4,2012-10-01 00:57:24.351689+00:00,7.588919,82.706331,0.390625,6,42.494557,-124.657684,0.390625,4,111915,-0.655013,G25B,P,1.349053e+09,2012-10-01 00:57:24.351689+00:00
2,2,4,2012-10-01 00:57:24.351689+00:00,7.588919,82.706331,0.390625,6,42.494557,-124.657684,0.390625,4,4333,0.254098,J25B,P,1.349053e+09,2012-10-01 00:57:24.351689+00:00
3,3,4,2012-10-01 00:57:24.351689+00:00,7.588919,82.706331,0.390625,6,42.494557,-124.657684,0.390625,4,150614,-1.278737,J25B,S,1.349053e+09,2012-10-01 00:57:24.351689+00:00
4,4,4,2012-10-01 00:57:24.351689+00:00,7.588919,82.706331,0.390625,6,42.494557,-124.657684,0.390625,4,170582,1.093668,J33B,S,1.349053e+09,2012-10-01 00:57:24.351689+00:00


In [47]:
# Get the picks in my catalog that occurred on 10/14/2012 and match with the three events in Morton's catalog
t1 = events_morton_2021014.iloc[0]['datetime']
event_morton_2021014_023944 = mycatalog.loc[(mycatalog['datetime'] > t1 - pd.Timedelta(5,'minutes')) & (mycatalog['datetime'] < t1 +pd.Timedelta(5,'minutes')) ]

t2 = events_morton_2021014.iloc[1]['datetime']
event_morton_2021014_030801 = mycatalog.loc[(mycatalog['datetime'] > t2 - pd.Timedelta(5,'minutes')) & (mycatalog['datetime'] < t2 +pd.Timedelta(5,'minutes')) ]

t3 = events_morton_2021014.iloc[2]['datetime']
event_morton_2021014_164833 = mycatalog.loc[(mycatalog['datetime'] > t3 - pd.Timedelta(5,'minutes')) & (mycatalog['datetime'] < t3 +pd.Timedelta(5,'minutes')) ]

In [48]:
# Print the events in my catalog that match with the event in Morton's catalog
event_morton_2021014_023944

,Unnamed: 0,idx,time,x,y,z,picks,latitude,longitude,depth,event_idx,pick_idx,residual,station,phase,time_pick,datetime
2285,2285,1568,2012-10-14 02:39:43.926355+00:00,15.761602,-168.807655,49.609375,12,40.229798,-124.564802,49.609375,1568,55255,0.724059,FS09B,P,1.350182e+09,2012-10-14 02:39:43.926355+00:00
2286,2286,1568,2012-10-14 02:39:43.926355+00:00,15.761602,-168.807655,49.609375,12,40.229798,-124.564802,49.609375,1568,191232,-0.314155,FS09B,S,1.350182e+09,2012-10-14 02:39:43.926355+00:00
2287,2287,1568,2012-10-14 02:39:43.926355+00:00,15.761602,-168.807655,49.609375,12,40.229798,-124.564802,49.609375,1568,143917,-0.464841,FS13B,P,1.350182e+09,2012-10-14 02:39:43.926355+00:00
2288,2288,1568,2012-10-14 02:39:43.926355+00:00,15.761602,-168.807655,49.609375,12,40.229798,-124.564802,49.609375,1568,268393,0.010119,FS13B,S,1.350182e+09,2012-10-14 02:39:43.926355+00:00
2289,2289,1568,2012-10-14 02:39:43.926355+00:00,15.761602,-168.807655,49.609375,12,40.229798,-124.564802,49.609375,1568,140796,-0.523190,FS17B,P,1.350182e+09,2012-10-14 02:39:43.926355+00:00
2290,2290,1568,2012-10-14 02:39:43.926355+00:00,15.761602,-168.807655,49.609375,12,40.229798,-124.564802,49.609375,1568,266057,-0.085016,FS17B,S,1.350182e+09,2012-10-14 02:39:43.926355+00:00
2291,2291,1568,2012-10-14 02:39:43.926355+00:00,15.761602,-168.807655,49.609375,12,40.229798,-124.564802,49.609375,1568,142618,0.166459,FS20B,P,1.350182e+09,2012-10-14 02:39:43.926355+00:00
2292,2292,1568,2012-10-14 02:39:43.926355+00:00,15.761602,-168.807655,49.609375,12,40.229798,-124.564802,49.609375,1568,267331,0.804205,FS20B,S,1.350182e+09,2012-10-14 02:39:43.926355+00:00
2293,2293,1568,2012-10-14 02:39:43.926355+00:00,15.761602,-168.807655,49.609375,12,40.229798,-124.564802,49.609375,1568,747,-0.344330,G10B,P,1.350182e+09,2012-10-14 02:39:43.926355+00:00
2294,2294,1568,2012-10-14 02:39:43.926355+00:00,15.761602,-168.807655,49.609375,12,40.229798,-124.564802,49.609375,1568,147714,-0.366297,G10B,S,1.350182e+09,2012-10-14 02:39:43.926355+00:00


In [49]:
# Print the events in my catalog that match with the event in Morton's catalog
event_morton_2021014_030801

,Unnamed: 0,idx,time,x,y,z,picks,latitude,longitude,depth,event_idx,pick_idx,residual,station,phase,time_pick,datetime
2297,2297,1570,2012-10-14 03:08:01.434268+00:00,30.939441,-96.402113,49.609375,13,40.881398,-124.382923,49.609375,1570,191233,-0.045203,FS09B,S,1.350184e+09,2012-10-14 03:08:01.434268+00:00
2298,2298,1570,2012-10-14 03:08:01.434268+00:00,30.939441,-96.402113,49.609375,13,40.881398,-124.382923,49.609375,1570,143926,-0.550344,FS13B,P,1.350184e+09,2012-10-14 03:08:01.434268+00:00
2299,2299,1570,2012-10-14 03:08:01.434268+00:00,30.939441,-96.402113,49.609375,13,40.881398,-124.382923,49.609375,1570,268405,1.129425,FS13B,S,1.350184e+09,2012-10-14 03:08:01.434268+00:00
2300,2300,1570,2012-10-14 03:08:01.434268+00:00,30.939441,-96.402113,49.609375,13,40.881398,-124.382923,49.609375,1570,177473,1.024022,FS16B,S,1.350184e+09,2012-10-14 03:08:01.434268+00:00
2301,2301,1570,2012-10-14 03:08:01.434268+00:00,30.939441,-96.402113,49.609375,13,40.881398,-124.382923,49.609375,1570,140807,-0.749436,FS17B,P,1.350184e+09,2012-10-14 03:08:01.434268+00:00
2302,2302,1570,2012-10-14 03:08:01.434268+00:00,30.939441,-96.402113,49.609375,13,40.881398,-124.382923,49.609375,1570,266063,0.609988,FS17B,S,1.350184e+09,2012-10-14 03:08:01.434268+00:00
2303,2303,1570,2012-10-14 03:08:01.434268+00:00,30.939441,-96.402113,49.609375,13,40.881398,-124.382923,49.609375,1570,142621,-0.688690,FS20B,P,1.350184e+09,2012-10-14 03:08:01.434268+00:00
2304,2304,1570,2012-10-14 03:08:01.434268+00:00,30.939441,-96.402113,49.609375,13,40.881398,-124.382923,49.609375,1570,267335,-1.107447,FS20B,S,1.350184e+09,2012-10-14 03:08:01.434268+00:00
2305,2305,1570,2012-10-14 03:08:01.434268+00:00,30.939441,-96.402113,49.609375,13,40.881398,-124.382923,49.609375,1570,749,-0.769603,G10B,P,1.350184e+09,2012-10-14 03:08:01.434268+00:00
2306,2306,1570,2012-10-14 03:08:01.434268+00:00,30.939441,-96.402113,49.609375,13,40.881398,-124.382923,49.609375,1570,147716,-0.604589,G10B,S,1.350184e+09,2012-10-14 03:08:01.434268+00:00


In [50]:
# Print the events in my catalog that match with the event in Morton's catalog
event_morton_2021014_164833

,Unnamed: 0,idx,time,x,y,z,picks,latitude,longitude,depth,event_idx,pick_idx,residual,station,phase,time_pick,datetime
2399,2399,1632,2012-10-14 16:53:05.402505+00:00,-58.960066,-123.077839,37.890625,26,40.639663,-125.446993,37.890625,1632,55380,-0.022246,FS09B,P,1.350234e+09,2012-10-14 16:53:05.402505+00:00
2400,2400,1632,2012-10-14 16:53:05.402505+00:00,-58.960066,-123.077839,37.890625,26,40.639663,-125.446993,37.890625,1632,191333,-0.108078,FS09B,S,1.350234e+09,2012-10-14 16:53:05.402505+00:00
2401,2401,1632,2012-10-14 16:53:05.402505+00:00,-58.960066,-123.077839,37.890625,26,40.639663,-125.446993,37.890625,1632,144215,-0.188136,FS13B,P,1.350234e+09,2012-10-14 16:53:05.402505+00:00
2402,2402,1632,2012-10-14 16:53:05.402505+00:00,-58.960066,-123.077839,37.890625,26,40.639663,-125.446993,37.890625,1632,268535,-0.256534,FS13B,S,1.350234e+09,2012-10-14 16:53:05.402505+00:00
2403,2403,1632,2012-10-14 16:53:05.402505+00:00,-58.960066,-123.077839,37.890625,26,40.639663,-125.446993,37.890625,1632,35687,0.237993,FS16B,P,1.350234e+09,2012-10-14 16:53:05.402505+00:00
2404,2404,1632,2012-10-14 16:53:05.402505+00:00,-58.960066,-123.077839,37.890625,26,40.639663,-125.446993,37.890625,1632,177663,0.480991,FS16B,S,1.350234e+09,2012-10-14 16:53:05.402505+00:00
2405,2405,1632,2012-10-14 16:53:05.402505+00:00,-58.960066,-123.077839,37.890625,26,40.639663,-125.446993,37.890625,1632,141098,0.522280,FS17B,P,1.350234e+09,2012-10-14 16:53:05.402505+00:00
2406,2406,1632,2012-10-14 16:53:05.402505+00:00,-58.960066,-123.077839,37.890625,26,40.639663,-125.446993,37.890625,1632,266201,-0.910331,FS17B,S,1.350234e+09,2012-10-14 16:53:05.402505+00:00
2407,2407,1632,2012-10-14 16:53:05.402505+00:00,-58.960066,-123.077839,37.890625,26,40.639663,-125.446993,37.890625,1632,142751,-0.516596,FS20B,P,1.350234e+09,2012-10-14 16:53:05.402505+00:00
2408,2408,1632,2012-10-14 16:53:05.402505+00:00,-58.960066,-123.077839,37.890625,26,40.639663,-125.446993,37.890625,1632,267449,-1.016373,FS20B,S,1.350234e+09,2012-10-14 16:53:05.402505+00:00


In [51]:
client = WaveformClient()
client2 = Client("IRIS")

In [52]:
# Get the list of stations
station_list = ['FS09B',
 'FS13B',
 'FS16B',
 'FS17B',
 'FS20B',
 'G10B',
 'G17B',
 'G19B',
 'G25B',
 'G26B',
 'G27B',
 'J09B',
 'J25B',
 'J33B',
 'M09B',
 'M12B',
 'M14B']

In [53]:
mycatalog[mycatalog['idx'] == 9]

,Unnamed: 0,idx,time,x,y,z,picks,latitude,longitude,depth,event_idx,pick_idx,residual,station,phase,time_pick,datetime
6,6,9,2012-10-01 01:36:43.089506+00:00,-99.823478,21.733243,49.609375,6,41.939361,-125.953707,49.609375,9,154575,0.588473,FS16B,S,1.349055e+09,2012-10-01 01:36:43.089506+00:00
7,7,9,2012-10-01 01:36:43.089506+00:00,-99.823478,21.733243,49.609375,6,41.939361,-125.953707,49.609375,9,116106,-0.962265,FS17B,P,1.349055e+09,2012-10-01 01:36:43.089506+00:00
8,8,9,2012-10-01 01:36:43.089506+00:00,-99.823478,21.733243,49.609375,6,41.939361,-125.953707,49.609375,9,215133,0.268530,FS20B,S,1.349055e+09,2012-10-01 01:36:43.089506+00:00
9,9,9,2012-10-01 01:36:43.089506+00:00,-99.823478,21.733243,49.609375,6,41.939361,-125.953707,49.609375,9,134135,0.638193,M09B,P,1.349055e+09,2012-10-01 01:36:43.089506+00:00
10,10,9,2012-10-01 01:36:43.089506+00:00,-99.823478,21.733243,49.609375,6,41.939361,-125.953707,49.609375,9,104145,-0.595106,M12B,P,1.349055e+09,2012-10-01 01:36:43.089506+00:00
11,11,9,2012-10-01 01:36:43.089506+00:00,-99.823478,21.733243,49.609375,6,41.939361,-125.953707,49.609375,9,234957,0.062176,M12B,S,1.349055e+09,2012-10-01 01:36:43.089506+00:00


In [54]:
mycatalog['station'].unique()

array(['FS13B', 'G25B', 'J25B', 'J33B', 'M12B', 'FS16B', 'FS17B', 'FS20B',
       'M09B', 'G10B', 'G19B', 'FS09B', 'M14B', 'G17B', 'J09B', 'G26B',
       'G27B'], dtype=object)

In [55]:
distances

{'FS16B': 22.976654541948474,
 'FS17B': 23.99429363189918,
 'FS13B': 28.48602082567876,
 'M14B': 29.668964314057174,
 'FS09B': 34.47269178722089,
 'FS20B': 46.38911307982181,
 'G17B': 68.64657323532724,
 'G10B': 68.69248304206646,
 'G19B': 106.71142014127307,
 'G25B': 131.329973741835,
 'G26B': 136.14972293586126,
 'M12B': 160.93208595898352,
 'G27B': 168.18863508100011,
 'J09B': 267.56986944669507,
 'M09B': 390.59436557586747,
 'J25B': 414.49581671894094,
 'J33B': 485.2224023162967}

In [56]:
distances.keys()

dict_keys(['FS16B', 'FS17B', 'FS13B', 'M14B', 'FS09B', 'FS20B', 'G17B', 'G10B', 'G19B', 'G25B', 'G26B', 'M12B', 'G27B', 'J09B', 'M09B', 'J25B', 'J33B'])

In [57]:
idx = 21
otime = UTCDateTime(mycatalog['datetime'].unique()[idx])
distances = {}
for i, sta in enumerate(station_list):
    sta_inv = client2.get_stations(network = "7D", station = sta, channel = "?H?", 
                     starttime = otime - 1e4, endtime = otime + 1e4)[0][0]
    slat = sta_inv.latitude
    slon = sta_inv.longitude
    olat = mycatalog['latitude'].unique()[idx]
    olon = mycatalog['longitude'].unique()[idx]

    dis1 = locations2degrees(olat, olon, slat, slon)
    dist = degrees2kilometers(dis1)
    distances[sta] = dist
distances = {k: v for k, v in sorted(distances.items(), key=lambda item: item[1])}
# print(distances.keys())
print(otime)
plt.figure(figsize=(15,10), dpi=150)
for i, sta in enumerate(distances.keys()):
    st = client.get_waveforms(network="7D", station=sta, channel="?H?", year=2012, month=10, day=14)
    st = obspy.Stream(filter(lambda st:st.stats.sampling_rate>10, st))
    st.filter(type='bandpass',freqmin=4,freqmax=15)
    print(st)
    trim_st = st.copy().trim(starttime = otime - 30, 
                endtime = otime + 60)
    trim_st = trim_st.normalize()
    offsets1  = np.arange(0,len(trim_st))
    offsets2 = np.arange(0,len(station_list)* (len(trim_st)),len(trim_st))

    for ii in range(len(trim_st)):
        plt.plot(trim_st[ii].times(type='utcdatetime'), trim_st[ii].data+offsets1[ii]*0.5+offsets2[i], 
                 color = 'gray', alpha=0.7, lw=0.5)    
    time_pick = [[x['time_pick'], x['phase']] for _, x in mycatalog[mycatalog['idx'] == idx].iterrows() 
                 if x['station'] == sta]
    if len(time_pick) > 0:
        for p in time_pick:
            if p[1] == 'P':
                plt.vlines(p[0], offsets1[ii]*0.5+offsets2[i]-1, 
                                 offsets1[ii]*0.5+offsets2[i]+1, color='r')
            else:
                plt.vlines(p[0], offsets1[ii]*0.5+offsets2[i]-1, 
                                 offsets1[ii]*0.5+offsets2[i]+1, color='b')

2012-10-02T04:46:27.808788Z
3 Trace(s) in Stream:
7D.FS16B..HH1 | 2012-10-14T00:00:00.004500Z - 2012-10-14T23:59:59.996500Z | 125.0 Hz, 10800000 samples
7D.FS16B..HH2 | 2012-10-14T00:00:00.004500Z - 2012-10-14T23:59:59.996500Z | 125.0 Hz, 10800000 samples
7D.FS16B..HHZ | 2012-10-14T00:00:00.004500Z - 2012-10-14T23:59:59.996500Z | 125.0 Hz, 10800000 samples


ZeroDivisionError: division by zero

<Figure size 2250x1500 with 0 Axes>

In [62]:
idx = 9
otime = UTCDateTime(mycatalog['datetime'].unique()[idx])
distances = {}
for i, sta in enumerate(station_list):
    sta_inv = client2.get_stations(network = "7D", station = sta, channel = "?H?", 
                     starttime = otime - 1e4, endtime = otime + 1e4)[0][0]
    slat = sta_inv.latitude
    slon = sta_inv.longitude
    olat = mycatalog['latitude'].unique()[idx]
    olon = mycatalog['longitude'].unique()[idx]

    dis1 = locations2degrees(olat, olon, slat, slon)
    dist = degrees2kilometers(dis1)
    distances[sta] = dist
distances = {k: v for k, v in sorted(distances.items(), key=lambda item: item[1])}
# print(distances.keys())
print(otime)
plt.figure(figsize=(15,10), dpi=150)
for i, sta in enumerate(distances.keys()):
    st = client.get_waveforms(network="7D", station=sta, channel="?H?", year=2012, month=10, day=14)
    st = obspy.Stream(filter(lambda st:st.stats.sampling_rate>10, st))
    st.filter(type='bandpass',freqmin=4,freqmax=15)
    print(st)
    copy_st = st.copy()
    print(copy_st)
    trim_st = st.copy().trim(starttime = otime - 30, 
                endtime = otime + 60)
    print(trim_st)
    trim_st = trim_st.normalize()
    offsets1  = np.arange(0,len(trim_st))
    offsets2 = np.arange(0,len(station_list)* (len(trim_st)),len(trim_st))

    for ii in range(len(trim_st)):
        plt.plot(trim_st[ii].times(type='utcdatetime'), trim_st[ii].data+offsets1[ii]*0.5+offsets2[i], 
                 color = 'gray', alpha=0.7, lw=0.5)    
    time_pick = [[x['time_pick'], x['phase']] for _, x in mycatalog[mycatalog['idx'] == idx].iterrows() 
                 if x['station'] == sta]
    if len(time_pick) > 0:
        for p in time_pick:
            if p[1] == 'P':
                plt.vlines(p[0], offsets1[ii]*0.5+offsets2[i]-1, 
                                 offsets1[ii]*0.5+offsets2[i]+1, color='r')
            else:
                plt.vlines(p[0], offsets1[ii]*0.5+offsets2[i]-1, 
                                 offsets1[ii]*0.5+offsets2[i]+1, color='b')

2012-10-01T17:18:00.934549Z
3 Trace(s) in Stream:
7D.G10B..BH1 | 2012-10-14T00:00:00.004647Z - 2012-10-14T23:59:59.984647Z | 50.0 Hz, 4320000 samples
7D.G10B..BH2 | 2012-10-14T00:00:00.004647Z - 2012-10-14T23:59:59.984647Z | 50.0 Hz, 4320000 samples
7D.G10B..BHZ | 2012-10-14T00:00:00.004647Z - 2012-10-14T23:59:59.984647Z | 50.0 Hz, 4320000 samples
3 Trace(s) in Stream:
7D.G10B..BH1 | 2012-10-14T00:00:00.004647Z - 2012-10-14T23:59:59.984647Z | 50.0 Hz, 4320000 samples
7D.G10B..BH2 | 2012-10-14T00:00:00.004647Z - 2012-10-14T23:59:59.984647Z | 50.0 Hz, 4320000 samples
7D.G10B..BHZ | 2012-10-14T00:00:00.004647Z - 2012-10-14T23:59:59.984647Z | 50.0 Hz, 4320000 samples
0 Trace(s) in Stream:



ZeroDivisionError: division by zero

<Figure size 2250x1500 with 0 Axes>